# Working with XGBoost in Julia
## A Quick Little Example from a Julia Newbie

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Julia_prog_language.svg/320px-Julia_prog_language.svg.png)

I have seen a rising interest in the new programming language *Julia* for use in data science since it released its 1.0 version a few months ago in August. Obviously, Kaggle does not have support for Julia yet, but I know there are some like me who are interested in the language and its current capabillities! 

For those who may be interested in doing this competition with the language (whether seriously or whether to just try out *Julia*), here is a working example of XGBoost used in the Julia language. I managed to advance quite a bit in the leaderboard from my initial score using this code without adjusting that much about the model besides the number of rounds. 

Obviously, this code has to be run in a Julia environment to see results... but I wanted to provide it here. 

In [ ]:
%%script false

# libraries used
# --------------
using XGBoost
using DataFrames
using CSV 
using StatsBase
using DelimitedFiles

# initial setup
@info "Set Working Directory..."
cd(joinpath(homedir(), "Downloads/santander-customer-transaction-prediction"))

function read_data()
    @info "Input Data..."
    train_df = CSV.read("train.csv")
    test_df = CSV.read("test.csv")
    return(train_df, test_df)
end

train_df, test_df = read_data()

idx = [c for c in names(train_df) if c != :ID_code && c != :target]

# XGBoost
# -------
num_rounds = 500

# define train sets 
# -----------------
train_x = convert(Array{Float32}, train_df[idx])
train_y = convert(Array{Int32}, train_df[:target])

# define test set
# ---------------
test_x = convert(Array{Float32}, test_df[idx])

dtrain = DMatrix(train_x, label = train_y)

boost = xgboost(dtrain, num_rounds, eta = .03, objective = "binary:logistic")

prediction = XGBoost.predict(boost, test_x)

prediction_rounded = Array{Int64, 1}(map(val -> round(val), prediction))

sub = hcat(test_df[:ID_code], prediction)
sub = DataFrame(sub)

# clean up submission
# -------------------
rename!(sub, :x1 => :ID_code)
rename!(sub, :x2 => :target)

CSV.write("predictions.csv", sub)


I'm not all that familiar with Julia, but this was a fun little exercise in playing around with learning syntax and dealing with some of the constraints of Julia (such as the dataframe "type" not playing well with other packages just yet). You'll notice that the code is pretty similar to Python or R as far as XGBoost is concerned. You'll also notice that I didn't do any feature engineering, EDA, etc. These are all reasonably able to be accomplished in Julia, but I haven't quite learned how to do all those things yet since I'm still attached at the hip to R!

Hopefully this code helps someone out who might be looking to play around with *Julia* ! Though I'll probably be sticking to R and Python for now since there's no real reason for me to switch, *Julia* is definitely on my radar. 